In [1]:
from wilcoxon import *
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
def koreanLesson(link):
    site = spiderman.website(link)
    title = re.findall('Lesson \d+',site.html(class_='titlebar-title')[0].get_text())[0]
    words = {}
    for collapse in site.html(class_='collapseomatic'):
        word = collapse.find_previous('a').get_text()
        words.update({word:{}})
        try:
            words[word]['Meaning'] = collapse.get_text()
        except: continue
        try:
            maybe_common = collapse.find_next(string=re.compile('Common'))
            if maybe_common.find_next('a') == collapse.find_next('a'):
                words[word]['Common'] = maybe_common.parent.get_text().split('\n')[1:]
        except: continue
        try:
            maybe_examples = collapse.find_next(string=re.compile('Examples'))
            if maybe_examples.find_next('a') == collapse.find_next('a'):
                ## removed \n
                # words[word]['Examples'] = [i.replace('\n',' ') for i in re.split('\n',maybe_examples.parent.get_text(),maxsplit=1)]
                words[word]['Examples'] = [i.replace('\n',' ') for i in re.split('\n(?!=)',re.split('\n',maybe_examples.parent.get_text(),maxsplit=1)[1])]
        except: continue

    if len(words) == 0:
        for i in site.html(string=re.compile('^ = [a-z]')):
            words.update({i.find_previous('a').get_text():{'Meaning':i.split('=')[1].strip()}})
        
    if len(words) == 0:
        for i in re.findall('(.* = .{1,40})\n',site.html.get_text()):
            words.update({i.split(' = ')[0]:i.split(' = ')[1]})

    contents = {}
    hrefs = [x['href'][1:] for x in site.html(href=re.compile('^#\d'))]
    for href in hrefs:
        try:
            topic = site.html(attrs = {'name':href})[0].find_next('p',string=re.compile('\w'))
            topicText = topic.get_text()
            contents.update({topicText:{}})
            contents[topicText]['Link'] = link + '#' + href
            try:
                sentence = topic.find_next('a')
                sentenceText = sentence.get_text()
                contents[topicText]['Sentence'] = sentenceText
                meaningText = sentence.find_next('p').get_text()
                contents[topicText]['Meaning'] = meaningText
            except: continue
            numColon = 0
            intro = ''
            sentences = []
            currentTag = topic.find_next('p')

            while numColon == 0:
                try:
                    if ':' in currentTag.get_text():
                        numColon += 1
                        if '.' in currentTag.get_text():
                            intro += ' ' + currentTag.get_text()
                    else: intro += ' ' + currentTag.get_text()
                    currentTag = currentTag.find_next('p')
                except: break

            contents[topicText]['Intro'] = '.'.join(re.split('\.[^\.]',intro)[:5])
        except Exception as e: print(e)
    return {title:{'Link':link,'Words':words,'Contents':contents}}
        

In [37]:
def koreanLessonFull():
    site = spiderman.website('https://www.howtostudykorean.com/other-stuff/lesson-list/')
    links = []
    for i in site.html(string=re.compile('Lesson ')):
        try: 
            link = i.find_previous('a')['href']
            if 'other-stuff' not in link:
                links.append(link)
        except: continue

    lessons = {}
    for link in tqdm(links):
        try:lessons.update(getLesson(link))
        except Exception as e:
            print(link)
            print(e)
    return lessons

  1%|          | 1/135 [00:02<04:55,  2.20s/it]

https://www.howtostudykorean.com/unit0/unit0lesson1/
list index out of range


  1%|▏         | 2/135 [00:04<04:45,  2.14s/it]

https://www.howtostudykorean.com/unit0/0-lesson-2/
list index out of range


  2%|▏         | 3/135 [00:05<04:21,  1.98s/it]

...

 98%|█████████▊| 132/135 [04:03<00:05,  1.86s/it]

list index out of range


 99%|█████████▊| 133/135 [04:05<00:03,  1.86s/it]

list index out of range


100%|██████████| 135/135 [04:09<00:00,  1.85s/it]


In [5]:
words = pd.DataFrame(columns = ['Word','Meaning','Common','Examples','Lesson'])
content = pd.DataFrame()
failures = []
for lesson in lessons:
    lessonContent = lessons[lesson]
    try:
        dfToAdd = pd.DataFrame(lessonContent['Words']).T.reset_index().rename(columns = {'index':'Word'})
        dfToAdd['Lesson'] = lesson
        dfToAdd['Link'] = lessonContent['Link']
        words = pd.concat([words,dfToAdd]).reset_index(drop=True)
        words = words[words['Word'].apply(lambda x: 'Lesson' not in x)]
    except Exception as e: 
        dictToAdd = {}
        failureWords = lessonContent['Words']
        for word in failureWords:
            dictToAdd[word] = {'Meaning':failureWords[word]}
        dfToAdd = pd.DataFrame(dictToAdd).T.reset_index().rename(columns = {'index':'Word'})
        dfToAdd['Lesson'] = lesson
        dfToAdd['Link'] = lessonContent['Link']
        words = pd.concat([words,dfToAdd])
        words = words[words['Word'].apply(lambda x: 'Lesson' not in x)]
    
words = words[words['Word'].apply(lambda x: ' ' not in x)]
words = words.explode('Common')

for common in words['Common']:
    try:
        splitted = common.split(' = ')
        commonDf = pd.DataFrame(splitted).T
        commonDf.columns = ['Word','Meaning']
        words = pd.concat([words,commonDf])
    except: continue
       
words['Temp Examples'] = words['Examples'].str.join(';')
words = words.drop_duplicates(['Word','Temp Examples'])
words['Examples'] = words['Examples'].str.get(0)
words[['Sentences','Translation']] = words['Examples'].str.split(' = ',expand=True)[[0,1]]

from hangul_romanize import Transliter
from hangul_romanize.rule import academic

transliter = Transliter(academic)
def koreanRomanizer(x):
    try: return transliter.translit(x)
    except: return x
    
words['Romanized'] = words['Word'].apply(koreanRomanizer)
words = words[['Word','Romanized','Meaning','Sentences','Translation','Lesson','Link']].reset_index(drop=True)


In [41]:
pd.DataFrame(forData).T.reset_index(drop=False).rename({'index':'Text'},axis=1).to_clipboard()

In [38]:
forData = {}
for lesson in lessons:
    forData.update(lessons[lesson]['Contents'])

In [174]:
words.to_clipboard()

In [178]:
ja = 
noun + のように means like, similar to, in the way of
noun 1 + のほうが + noun 2 + より + adjective means noun 1 is more (adjective) than noun 2. This structure is used to make comparisons. The first part, のほうが, conveys the idea that the noun it follows is  'better' or 'worse', more or less etc, depending on the context. The second part, より, means 'than' or 'rather than'.
noun phrase + があります means something exists. It is used for inanimate objects, not for living things.
noun phrase + がいます means something exists. It is used for living things.
On its own, the て form of a verb can be used as a simple joining form, or as an imperative.However, the て form is the verb form used in many other constructions, so it is a key form to learn how to make.To create the て form, replace the た of the plain past form with て.
One of the most common patterns in Japanese is ""X is Y"". 「は」is the topic marker, placed immediately after the topic of the sentence.「です」is the ます (normal polite) form of the verb 'to be', so here it means 'is'.
plain form clause or noun + にしては means considering that ~, yet etc.Note: in some sentences it is easy to confuse this construction with ~ていけない (must not ~) applied to する. The て form of する is して, and the 'must not' construction often has a ’は’ inserted between the て and the い, so it becomes 〜してはいけない. For example, バカにしてはいけない means '(you) must not make fun of', rather than 'considering that (he's) an idiot'!
plain past verb + おり折に or noun + のおり折に is another expression meaning 'when', but in this case it is used to denote a special occasion or an unusual circumstance. 折 is more formal than とき時, and it is only used when referring to positive events or situations.
plain present clause + ことはない means there is no need to ~, ~ never happens.This construction is similar to ことがない, but typically accompanies the present tense. 
subject / topic + には + determiner + noun + があるmeans There is / are (specified thing/s) in (place / location), or(something) has (specified thing/s)この街にはたくさんの自動車メーカがあります。There are many car manufacturers in this town.この表現には三つの意味があります。This expression has 3 meanings.
The causative form of a verb means to make the verb happen. For ichidan (る) verbs, the causative is formed by replacing the る with させる: た食べる to eat ➜ た食べさせる make eat
The conditional form of a verb means 'if (verb)'. The conditional is formed by replacing the う sound of the dictionary form with the equivalent え and then adding ば. So for example:よむ read ➜ よめば if I readたべる eat ➜ たべれば if I eatThis 'ば form' can actually be used with adjectives and nouns. In the case of い adjectives, the い is replaced with ければ:よい good ➜ よければ if it's goodない not ➜ なければ if it's notFor nouns and な adjectives, just add であれば (which comes from である, a form of 'is'):にちようび日曜日 Sunday ➜ にちようび日曜日であれば if it's Sundayしず静か quiet ➜ しず静かであれば if it's quiet
The expression そうです can be used in two similar but slightly different ways: to say that something seems a certain way, or to suggest that you heard that something is the case. It can also be used as a standalone expression of agreement.seems like ~, appears ~adjective + そうです or verb stem + そうですmeans seems like (something).One very common example is おいしそうです, meaning 'looks tasty', which is almost a set phrase for example when seeing food prepared by a host.For い adjectives, the い is dropped before the そうです. たか高そうです   (that) seems expensiveな adjectives are used as is:しんせつ親切そうです  (he/she) seems kindFor verbs, the stem is used:かれ彼はお起きそうです he looks like he's about to wake uphearsay, I heard that ~A very similar construction is used to express 'hearsay', i.e. 'I heard that ~'. In this case, the form is:verb plain form or verb plain past + そうですかれ彼はつか疲れているそうです He seems tiredあめ雨がふ降ったそうです They say it rainedWhen used with adjectives, a な adjective is followed by だ, and an い adjective keeps its い:しんせつ親切だそうです  I heard he/she's kindたか高いそうです   I heard that's expensiveYou may occasionally see だ after an い adjective, but this is to convey additional nuance, and would be incorrect as a form of the そうです construction itself. (For example, 嬉しいだそうよ might be a response to a question such as お父さんは気に入ってくれた？, in which case it would mean that the speaker is reporting that the the speaker thinks the father said '嬉しいだ', not that the speaker heard that the subject is 嬉しい.)standalone expressionそうです can also be used as a standalone expression which indicates agreement / confirmation:「そうです。」or 「そうですね。」
The Japanese past tense is used for two common English tenses - the simple past (I came, I saw, I conquered etc), as well as the present perfect (I have read, I have understood etc).The plain past is easy to form for ichidan verbs - just replace the る with た. For example: た食べる (eat) ➜ た食べた ateFor godan verbs it's a little more complicated - the past ending depends on the consonant used in the plain form:  Plain form Plain past form
The negative plain form of the verb followed by でください is used to request (somebody) not to do something, i.e. 'please don't ~'. For more information on the negative plain form, see Japanese verb conjugation.
The negative volitional expresses the idea of wanting something not to happen, or being determined for something not to happen. It evokes the opposite of the normal volitional (which means 'let's ~' or 'shall we'), and even more strongly ('determined not to do'). In Japanese, it is formed by adding まい to the plain form of ごだん verbs, and the verb stem of いちだん verbs (although the plain form is also sometimes used).For example:   み見る (see) み見まい (determined not to see)
The particle と is used to join multiple nouns in a list.リンゴとオレンジはくだもの果物です   apples and oranges are (both) fruits
The particle は is placed after the subject / topic of a sentence.Important note: although hiragana は (ha) is used for this particle, it is actually pronounced 'wa'.
The particles で and に can both be used as location and time indicators, and the difference between the two can sometimes be subtle.For location, で is used to mark the location where an event occurred (ie, a focus on action). に is used to indicate object locations, destinations or directions (in the absence of a specific 'event'). For example:がっこう学校でべんきょう勉強しました  I studied at schoolがっこう学校にほん本がたくさんあります  there are many books in schoolFor time, で indicates the (range of) time within which an action takes place, whereas に indicates the specific time when an action is performed. For example:ほん本を３じかん時間でよ読み終[お]わった   I finished reading the book in three hoursほん本を３じ時によ読みはじ始めた   I began reading the book at 3 o'clockJust as with English, sometimes these particles are roughly equivalent. In many cases, both can be translated with the English 'in', and even in Japanese, there are situations where using either can work. However, the emphasis is usually subtly different.The following are some more subtle examples. Here, in the second example, the wall is the location where the event of writing takes place:壁に書く  write on a wall壁で書く  write at a wallHere, the different particles imply a different object of focus in the sentence:て手でい入れる   I put it in by handて手にい入れる   I put it in my handIn this pair, the first sentence simply describes when you will perform the action, whereas the second implies that you could do it now, but instead will perform it later:あと後でやります  I will do it laterあと後にします   I will do it at a later time = I will put it off till laterIn this example, the に implies that the room you rented is in Kyoto, whereas the で implies that the action of renting took place in Kyoto:きょうと京都にへや部屋をか借りた I rented a room in Kyotoきょうと京都でへや部屋をか借りた I rented a room (while I was) in KyotoIn this example, the English word 'in' is used in both cases. However, both English and Japanese would use word order to indicate the different emphasis:にほん日本でこめ米はたか高いです   in Japan, rice is expensiveにほん日本にたか高いこめ米があります   there is expensive rice in JapanOne example that is commonly confusing for learners is the use of に with す住む. Although in English we might think of 'the act of living', in Japanese it is not an action but a state of affairs, and thus takes に:とうきょう東京にす住んでいます I live in Tokyo
The passive causative form of a verb means that something 'is made' to happen.For both godan and ichidan verbs, the passive causative is made by first making the causative, then replacing the る with られる. For example:  た食べる eat ➜ た食べさせる make eat ➜ た食べさせられる is made to eat
The plain form of all Japanese verbs ends with the vowel sound 'u'. In the case of Godan verbs, the u sound may be prefixed by a consonant, or it may stand on its own. Ichidan verbs always end in る.The plain form of a verb is the best form to learn. It's the form found in dictionaries, the form used most commonly in Japanese (since it's used in informal speech and in all relative clauses). Plus, if you know the plain form, and the group it belongs to, you can create any conjugated form.
The plain past form of a verb + ことがあります means 'having done something previously', or 'having had the experience of doing (something)'. For example: い行ったことがあります   I have been  (from 行く = いく, go)
The potential form of a verb expresses the ability to do the action the verb describes. Where in English 'can' or similar words are added to a verb to express ability ('I can run'), in Japanese the verb itself is modified.The potential form is formed as follows:For ichidan (る) verbs, replace the る with られる. For example: たべる eat ➜ たべられる can eatFor godan (う) verbs, replace the vowel in the last sound with the equivalent sound containing え, then add る. For example: よむ read ➜ よめる can read, はなす speak ➜ はなせる can speakThere are a handful of irregular forms. For example: する do ➜ できる can do, くる come ➜ こられる can come, ある exist ➜ ありえる can existThe potential form acts as a standard ichidan (る) verb. See our verb conjugation overview for more information. Typically, the potential form only makes sense with transitive verbs.
The term 'verb stem', also known as 'masu stem', in Japanese grammar refers to the base part of the verb, and is used in many different constructions. The easiest way to identify the stem of a verb is to remove the 'masu/ます' when the verb is in its masu/ます form. The part of the verb that remains is the verb stem. For example:dictionary formmasu formstemたべる (eat)たべますたべいく (go)いきますいきよむ (read)よみますよみたべ、いき、よみ  are all examples of the verb stem.In Japanese, the verb stem is known as the れんようけい連用形 (which means 'continuative form'). 
The topic marker は, in both English and Japanese, is actually pronounced 'wa'. Elsewhere the hiragana symbol は is pronounced 'ha'. This historical quirk is the result of slight changes in pronunciation of Japanese words over hundreds of years, along with changes in how Japanese words are transliterated in kana.
The typical form 〜てください  used to ask someone to do something is 'casually polite'. ください is roughly equivalent to the English 'please', so by including it, any sentence is not unpolite. However, it is not especially polite - ください is closer to a demand than an optional request. An English sentence with an equivalent 'tone' may or may not include 'please'.The word ください can be made more polite in several ways. One option is to replace てください with てくださいませんか. This is a direct substitute, which would be too polite for family, colleagues etc, but would be appropriate when addressing eg a teacher or someone senior.Another alternative would be to use the construction ていただけませんか. This may not be a direct substitute depending on the sentence, as the (implied) subject of the sentence is the speaker (where for ください the implied subject is the listener).Yet another alternative, politest of the options here, would be to use the construction 〜ようおね願いもう申しあ上げます (literally, I humbly entreat you to ~).However, in addition to changing the form of 'please', to make the sentence consistent, it would also be necessary to improve the politeness level of what is being asked for. For example, the following are all variations of 'please don't use it':つか使わないで下さい (casually polite)おつか使いにならないでください (the honorific verb phrase makes this polite in informal situations)ごしよう使用にならないでください (use of the term 使用する is more formal than 使う)ごしよう使用にならないようおねが願いもう申しあ上げます
The volitional form means 'let us ~', 'shall we ~' The plain volitional of a godan verb is formed by replacing the う in the plain form with おう. For example, いく (go) becomes いこう (let's go). For an ichidan, the る is replaced with よう, so for example たべる (eat) becomes たべよう (let's eat).The volitional forms of the two irregular verbs, する (do) and くる (come) are しよう and こよう.The polite volitional form for both ichidan and godan verbs is formed by taking the polite form and replacing the ます with ましょう. So for example たべます (eat, polite) becomes たべましょう (let's eat).
The volitional form of a verb plus としない means 'not trying to do ~'. For example:はなそうとしない not trying to talkしようとしない not trying to do, not making an effort to do
The ます (masu) form of a verb, also known as the polite form, is the form used in normal polite conversation, for example between two strangers of similar status. The ます form of godan verbs is created by taking the dictionary form and replacing the う with います. For ichidan verbs, the final る is replaced with ます.  Ichidan たべる たべます
This verb conjugation allows the speaker to express 'I want to ...'.です follows the conjugation, but is often omitted during informal interactions.Once たい has been added to the verb stem, the combination functions as an い adjective, and can be used to make all the standard forms of an い adjective.
time period + ぶ振り indicates that something hasn't occurred in the time interval specified. The English equivalent might be 'hasn't happened in ~ years' etc.
To create the negative plain form, replace the る of an ichidan verb with ない, and the う sound of a godan verb with あない.  Ichidan たべる たべない
Use to make suggestions and give advice - (I / you / she etc) shouldn't ~, It's better if (I / you /she etc) don't ~. 
Used after a place, indicates that an action will or has taken place there.
Used to make suggestions or to show willingness to do something. The English equivalent is typically ""let's ~"". For example:い行く ➜ い行きましょう  let's go
Used to mark the direct object of a sentence. Placed after the object and before the verb.
Used to request or politely order someone to do something.
Used to show current location. Placed after the location word or phrase.For example:なかに  insideIn this example, なか is the location, and に is required to mark it as such.
verb A plain form + ついでに + verb B means while doing A, do B at the same time
verb negative て form + もいい means ok not to do or you don't have to 
verb plain form + つもりです is used to express intention: I/you/we etc. intend/plan to ...
verb plain form + ようだ or ようです (at the end of a sentence) means appears to be ~  かんしん関心がある (has an interest) かんしん関心があるようだ (appears to have an interest)
verb plain negative + で + clause means do (clause) without doing (verb)verb plain negative + で is exactly the same as the negative imperative ('don't do').
verb plain negative + で is the negative imperative - it means do not (do something)The negative imperative can be made somewhat more polite by adding ください.
verb plain past + ほうがいいです is used to give advice or make suggestions. 'I / you / we etc should do ~', 'It would be better if you ~'.For example:そのほん本をよ読んだほうがいいですよ   you'd better read that book!
verb plain past form + こととて or noun + のこととて means that something is the case directly as a result of the preceding expression. This is not an expression that would be heard in speech; it is a more formal usage that might be found in a letter.こととて can also be written 事とて.
verb stem + あげる／あがる is used to indicate that an action has been or will be completed. あげる is used to indicate a transitive context (so and so has finished doing ~), whereas あがる is used to indicate an intransitive context (~ has come to an end).For example:しょくじ食事ができあがった   dinner is readyほん本を書き上げた   I finished writing the bookSeveral verbs are commonly used in this way:できあげる finish preparing (eg food)書き上げる finish writing築き上げる finish building立ち上げる start up, boot up縛り上げる tie upNote that 読み上げる does not follow quite the same pattern - it has the special meaning 'read aloud'.
verb stem + すぎる or い adjective stem + すぎる or な adjective + すぎる indicates too much of something. For example:  verb た食べる (eat) た食べすぎる eat too much
verb stem + っぱなし is used to describe something left in a state of being on or used. For example:  water running (みず水を) だ出しっぱな
verb stem + ながら is used to mean that while doing something, something else happened.
verb stem + なくて + はいけません means to have to do somethingThis form expresses obligation. Note that although this is based on the negative form in Japanese, the meaning in English is that you must do something.The plain form is 〜なくてはいけない, and the more informal version is 〜なくちゃいけない.
verb stem + なさい = do ~ !なさい is more polite than the standard imperative form, but more direct / stronger than ください.Examples:き来なさい!   come here!やめなさい!   stop it!
verb stem + にいきます is used to express the idea of going somewhere in order to do something. For example: 'go (somewhere) to study' = 'go (somewhere) for the purpose of studying'.
verb stem + にき来ます is used to express the idea of coming in order to do something. For example: 'come to study'.
verb stem + にくい means ~ is difficult
verb stem + ませんか is used to invite someone to (do) something, or to offer something to someone.
verb たら form + どう(ですか) is used for suggesting an action or giving advice, often with a slightly condescending tone. The たら form of a verb is the plain past form + ら.For example:くすり薬をの飲んだらどうですか I think you should take your medicineしず静かにしたらどうですか how about being quiet?じぶん自分でしら調べてみたらどうですか  why don't you try looking into it yourself?
verb て form + ある means that something has been completed or done.
verb て form + すみません means I'm sorry for doing ~.すみません on its own is considered 'casually polite'. In more polite speech, the すみません is usually followed by でした. In very casual speech, the form すまなかった or すまん may be used instead of すみません.
verb て form + はいけません means must not do somethingNote that although in Japanese this is the inverse of 〜なくてはいけません, which means 'must do' or 'have to do', the English is not directly the inverse, which would be 'do not have to do'.The plain form is 〜てはいけない, and the more informal version is 〜ちゃいけない.
verb て form + ほしい means want someone to do something
verb ば form + (same) verb plain form + ほど means the more (you do something) the more (something gets) 
verb ば form + よかった means 'I wish I'd done', or 'I should have done'. よかった is the past form of よい (good), so in English this construction would literally be translated as 'if I had done X, it would have been good'. Japanese does not have the pluperfect form (had done ~), and uses the simple past instead (did).
When a verb is in the て form, it functions as though it is followed by the conjunction 'and' in English.
When follows a specified day of the week means 'on that date'.
When placed after a location and before the verb, indicates destination.
When placed after a location and before the verb, indicates destination.
When placed after a specific time or time interval and before a verb, the particle に indicates something that occurs at that time.
When placed between two nouns, the particle の indicates that noun 1 possesses noun 2, or alternatively noun 2 belongs to noun 1.の 2 ➜ 2 of 1; 2 belongs to 1
When used to modify a verb, かた refers to the 'way of doing' something.
When using い言います (say) or い言いました (said) to report or quote speech, the particle と is placed before 言います. This expression is often written using hiragana only: 〜といいます (or the plain form という).
When から follows a verb or い adjective it translates to 'because' or 'so' in English. The information before から always expresses a reason.
When する, or the politer form します, is placed after a noun, the word becomes a verb.
When ですから follows a noun it means 'because, so, therefore'.
あくまでも + clause means 'to the utmost, definitely, extremely, to the nth degree, to the bitter end' etc. It is used to provide strong emphasis.The expression comes from あく meaning 'to be satisfied' and まで meaning 'until'. The も is optional, but usually included.
あの is a determiner which means 'that ~ over there'. Determiners are placed directly before the noun they specify.この and その are also determiners. この means ‘this’, eg: this book, this pencil. その means ‘that’, eg: that bag, that girl. あの and その are typically translated with the same English word (that), but usage in Japanese is determined by the (physical or metaphorical) distance to the noun from the speaker. あの is for things further from the speaker, and その is for things nearer to the speaker.  
おう is the plain volitional form for godan verbs.あそ遊ぶ play ➜ あそ遊びましょう let's play ➜ あそ遊ぼう  let's play (plain volitional form)かえ帰る return ➜ かえ帰りましょう let's return ➜ かえ帰ろう let's return (plain volitional form)
かもしれない can be added to the end of any sentence in place of the verb, and means may be, probably is, probably will.It is often used when describing future weather (since weather can't be predicted with absolute certainty). For example:あした明日はは晴れかもしれない  tomorrow may well be fineIn informal conversation, the しれない can be omitted, or replaced with しれん, especially when the user is implying that they are thinking through something on the spot, eg as in かもしれんなあ. In more polite conversation, the form かもしれません would be used.Note: the しれない comes from 知る (to know), so かも知れない and かも知れません are also common.
たかったです is the past form of 「〜たいです」(want to ~), meaning 'wanted to ~'. です follows the conjugation, but is often omitted during informal interactions.
って is a simple and casual substitute for the common expression という, which is used to quote speech within sentences. In casual speech って can also be used to end a sentence (which is uncommon with という).
でしょう means 'probably is', or 'might'. It is often used in rhetorical questions to solicit agreement, in a similar way to ですね.でしょう is the same politeness level as a ます verb form. The plain equivalent is だろう.
です appears at the end of a sentence, and means 'is', 'am' or 'are' depending on context.わたし私のなまえ名前はジョンです。  My name is John.これはほん本です。  This is a book.
ね (typically ですね) turns a statement into a 'tag' question, asking for agreement from the listener. The English equivalent is typically 'isn’t it?', 'aren’t they?', right? etc.
や is used to link items in a list, with the suggestion that the list is incomplete.
やくどし厄年, literally 'unlucky years', are the ages at which a person is most likely to experience personal misfortune, according to traditional Japanese belief. Years are counted by considering the year of birth #1, and adding one every time January 1st is reached. For men, years 25, 42 and 61 are considered most unlucky; for women, 19, 33 and 37. The origin of the superstition, and the precise reason for the years chosen, is unknown.
よ is often placed at the end of a sentence or after a word to emphasize an assertion, or in some cases to express opposition to another's assertion.
よう is the plain volitional form for ichidan verbs. (For godan verbs, the form is おう.)い行く go ➜ い行きましょう  let's go ➜ い行こう  let's go (plain volitional form)た食べる eat ➜ た食べましょう   let's eat ➜ た食べよう   let's eat (plain volitional form)
万 is a 'counter' marker, which when placed after a number, multiplies that number by ten thousand. To indicate exactly 10,000 objects (or a number between 10,000 and 19,999), the number one (一) is also required ('one ten thousand').

SyntaxError: invalid syntax (<ipython-input-178-c700f00d8fad>, line 1)

In [181]:
re.sub('(.+) .+ (.+)',r'\1\2','H I F')

'HF'